## 6.2 Univariable Plots
### 6.2.2 Histograms

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fires = pd.read_csv("data/VIIRSNDE_global2018312.v1.0.txt")
fires.head()

# print(plt.rcParams['figure.figsize'])
# plt.rcParams.update({'font.size': 16, 'figure.figsize':[12,6]})
# print(plt.rcParams['figure.figsize'])

In [ ]:
bins10MW = np.arange(0, 500, 10)

In [ ]:
plt.figure()
plt.hist(fires['frp(MW)'], bins=bins10MW)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(fires['frp(MW)'], bins=bins10MW)
ax.set_yscale('log')
ax.set_xlabel("Fire Radiative Power (MW)")
ax.set_ylabel("Counts")
plt.show()

### 6.2.2 Barplots

In [ ]:
ranges = [0.0, 100.0, 200.0, 1000.0]
categories = ['low', 'mid', 'high']

intensity = pd.cut(fires['frp(MW)'], bins=ranges, include_lowest=True, labels=categories)
intensity.head()

In [ ]:
intensity_counts = intensity.value_counts()

In [ ]:
plt.figure()
ax = plt.subplot(111)
ax.bar(x=intensity_counts.index, height=intensity_counts)
ax.set_yscale('log')
ax.set_xlabel("Fire Radiative Power (MW)")
ax.set_ylabel("Counts")
plt.show()

## 6.3 Two Variable Plots

In [ ]:
import pandas as pd
fname = 'data/campfire-gases-2018-11.csv'
trace_gases = pd.read_csv(fname)
trace_gases.columns

In [ ]:
trace_gases.sort_values(by='Time', inplace=True)
trace_gases.Time.values[0], trace_gases.Time.values[-1]

In [ ]:
trace_gases['Time'].head()

In [ ]:
fmt = '%Y-%m-%d %H:%M:%S'
trace_gases['Time'] = pd.to_datetime(trace_gases['Time'], format=fmt)
trace_gases['Time'].head()

In [ ]:
# Options to print figures into notebook/increase size
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams.update({'font.size': 16})

In [ ]:
plt.rcParams.update({'font.size': 16, 'figure.figsize': [12, 6]})

In [ ]:
ylabel = '$\mathregular{H_2}$O mixing ratio'

In [ ]:
mr = trace_gases['H2O_MR_850mb'].values
t = trace_gases['Time'].values

In [ ]:
import datetime
xlims = [datetime.date(2018, 11, 1), datetime.date(2018, 11, 30)]
ylims = [0, 0.015]
ylabel = '$\mathregular{H_2O}$ mixing ratio'

In [ ]:
plt.rcParams.update({'font.size': 16, 'figure.figsize':[12,6]})

fig = plt.figure()
ax = plt.subplot(111)

ax.scatter(t, mr, c='lightgray')
plt.xticks(rotation=35)
ax.set_xlim(xlims[0], xlims[1])
ax.set_ylim(ylims[0], ylims[1])

ax.set_ylabel(ylabel)
plt.show()

### 6.3.4 Line Plots

In [ ]:
daily_average = trace_gases.resample('D', on='Time').mean()
daily_average.head()

In [ ]:
daily_average = daily_average.reset_index()

In [ ]:
# trace_gases['Lon'] = trace_gases['Latitude'].round()
# groups = trace_gases.groupby(trace_gases['Lon'])
# daily_average = groups.mean()
# daily_average.head()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

#[ax.scatter(x, trace_gases['H2O_MR_850mb'][i], c='lightgray') for i, x in enumerate(trace_gases['Time'])]
ax.scatter(t, mr, c='lightgray')
ax.plot(daily_average['Time'], daily_average['H2O_MR_850mb'], linewidth=4)

ax.set_xlim(xlims[0], xlims[1])
ax.set_ylim(ylims[0], ylims[1])

ax.set_ylabel(ylabel)

fig.autofmt_xdate()

plt.show()

### 6.3.5 Adding data to an existing plot

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

ax.plot(daily_average['Time'], daily_average['H2O_MR_500mb'], label='500mb')
ax.plot(daily_average['Time'], daily_average['H2O_MR_850mb'], label='850mb')

ax.legend(loc='upper right')

ax.set_xlim(xlims[0], xlims[1])
ax.set_ylim(ylims[0], ylims[1])
ax.set_ylabel(ylabel)

fig.autofmt_xdate()

plt.show()

### 6.3.6 Plotting two side-by-side plots

In [ ]:
fig, ax = plt.subplots(ncols=2)

ax[0].set_title('Ozone')
ax[0].plot(daily_average['Time'], daily_average['O3_MR_500mb'])
ax[0].set_ylabel("Mixing Ratio (ppm)")

ax[1].set_title('Carbon Monoxide')
ax[1].plot(daily_average['Time'], daily_average['CO_MR_500mb'])

for ax0 in ax:
    ax0.set_xlim(xlims[0], xlims[1])

fig.autofmt_xdate()
plt.show()

### 6.3.7 Skew-T

In [ ]:
import xarray as xr
from metpy.units import units
import metpy.calc as mpcalc
from metpy.plots import SkewT

In [ ]:
fname = 'data/NUCAPS-EDR_v2r0_npp_s201903031848390_e201903031849090_c201903031941100.nc'
nucaps = xr.open_dataset(fname, decode_times=False)
list(nucaps.H2O_MR.attrs)

In [ ]:
nucaps.H2O_MR.units, nucaps.Temperature.units, nucaps.Pressure.units

In [ ]:
nucaps.Number_of_P_Levels

In [ ]:
profile = nucaps.sel(Number_of_CrIS_FORs=0)

In [ ]:
profile.H2O_MR.dims

In [ ]:
T = profile.Temperature.values.flatten()*units('K')
MR = profile.H2O_MR.values.flatten()*units('kg/kg')
P = profile.Effective_Pressure.values.flatten()*units('millibar')

In [ ]:
RH = mpcalc.relative_humidity_from_mixing_ratio(MR, T, P)

In [ ]:
RH[RH < 0] = 0
Td = mpcalc.dewpoint_from_relative_humidity(T, RH)

In [ ]:
fig = plt.figure(figsize=[8,8])
skew = SkewT(fig, subplot=111)

skew.plot(P, T, label='Temperature')
skew.plot(P, Td, linestyle='--', label='Dewpoint')

fig.legend()
plt.show()

## Three Variable Plots
### 6.4.1 Filled contour

In [ ]:
import h5py
import numpy as np

In [ ]:
fname = 'data/MOP03JM-201811-L3V95.6.3.he5' 
file_id = h5py.File(fname, 'r')

In [ ]:
#file_id.visit(print)
plt.rcParams.update({'font.size': 16, 'figure.figsize':[8,6]})

In [ ]:
file_id.visit(print)

In [ ]:
#file_id.visit(print)
grp_name="HDFEOS/GRIDS/MOP03/Data Fields/"

co = file_id[grp_name+"RetrievedCOTotalColumnDay"][:,:]
lat = file_id[grp_name+"Latitude"][:]
lon = file_id[grp_name+"Longitude"][:]

In [ ]:
missing = file_id[grp_name+"RetrievedCOTotalColumnDay"].attrs['_FillValue']
co[co== missing]=np.nan

In [ ]:
missing

In [ ]:
X_co, Y_co = np.meshgrid(lon, lat)

In [ ]:
co.shape, X_co.shape

In [ ]:
fig, ax = plt.subplots()

co_plot = ax.contourf(X_co, Y_co, co.transpose())

fig.colorbar(co_plot, orientation='horizontal', ax=ax)

ax.grid(True)

plt.show()

In [ ]:
fires["lon_bin"] = fires["Lon"].round(-1)
fires["lat_bin"] = fires["Lat"].round(-1)
fire_count = fires.groupby(["lon_bin", "lat_bin"]).size()

In [ ]:
fire_count = fire_count.reset_index(name="Count")
fire_count.head()

In [ ]:
fire_count_3D = fire_count.pivot(index='lat_bin', columns='lon_bin', values='Count')
fire_count_3D.head()

In [ ]:
X_fires, Y_fires = np.meshgrid(fire_count_3D.columns, fire_count_3D.index)

In [ ]:
from matplotlib import ticker, colors

In [ ]:
fig, ax = plt.subplots()

fire_plot = ax.contourf(X_fires, Y_fires, fire_count_3D, locator=ticker.LogLocator(), extend='both')
fig.colorbar(fire_plot, orientation='horizontal', shrink=0.5)

ax.scatter(fires['Lon'], fires['Lat'], s=0.5, c='black', alpha=0.1)

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

ax.grid(True)

plt.show()


### 6.4.2 Mesh plots

In [ ]:
fig, ax = plt.subplots()

co_plot = ax.pcolormesh(X_co, Y_co, co.transpose())
fig.colorbar(co_plot, orientation='horizontal')

ax.grid(True)

plt.show()

In [ ]:
fig, ax = plt.subplots()

im = ax.pcolormesh(X_fires, Y_fires, fire_count_3D, norm=colors.LogNorm(), vmin=1, vmax=1000)
fig.colorbar(im, orientation='horizontal', shrink=0.5, pad=0.25)

ax.grid(True)

plt.show()